In [1]:
#Use with python/3.6.0 and tensorflow/1.10

import math
import sys
from sklearn.linear_model import LogisticRegression
import random
import os
import csv
import json
from collections import defaultdict
import gensim
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegressionCV
import sklearn.metrics as metrics
from sklearn import metrics
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from glove import Corpus, Glove
#from bert_serving.client import BertClient
#bc = BertClient()
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
import openpyxl
 


from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout

from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import GridSearchCV, KFold

from sklearn.metrics import accuracy_score
from tensorflow.python.keras.constraints import maxnorm
from tensorflow.python.keras.callbacks import EarlyStopping
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_squared_error

lib_path = os.path.abspath(os.path.join('..', 'lib'))
sys.path.append(lib_path)

ds = "mimic"
data_path = "/Users/emily/Documents/SequentialPhenotypePredictor-master/Data/mimic_seq/"
#data_path = "/users/emily/Documents/mimic_seq/"
window=10
size=100
decay=5
skipgram=1
norm=False
random.seed(1)

# list for files
train_files = []
valid_files = []
full_data_files = []

ModuleNotFoundError: No module named 'bilm'

In [2]:
for i in range(7):
    train_files.append(data_path + 'test_'+str(i))
for i in range(7,10):
    valid_files.append(data_path + 'test_'+str(i))
for i in range(10):
    full_data_files.append(data_path + 'test_'+str(i))


In [3]:
diag_totals = defaultdict(lambda: 0)
diag_totals2 = defaultdict(lambda: 0)
diag_joined = defaultdict(lambda: 0)
diag_joined2 = defaultdict(lambda:0)
test = defaultdict(lambda:0)
sentences = []
final_dx_train = []
disease_prev = []
list_duplicates = []
diag = []


In [4]:
# Location of pretrained LM.  Here we use the test fixtures.
# Location of pretrained LM.  Here we use the test fixtures.
#datadir = '/users/emily/Documents/swp'
#vocab_file = '/users/emily/Documents/swp/vocab.txt'
#options_file = '/users/emily/Documents/swp/checkpoint/options.json'
#weight_file = '/users/emily/Documents/swp/swp_weights.hdf5'
datadir = '/home/egetzen/swp'
vocab_file = '/home/egetzen/swp/vocab.txt'
options_file = '/home/egetzen/swp/checkpoint/options.json'
weight_file = '/home/egetzen/swp/swp_weights.hdf5'
batcher = Batcher(vocab_file, 50)
context_character_ids = tf.placeholder('int32', shape=(None, None, 50))
bilm = BidirectionalLanguageModel(options_file, weight_file)
context_embeddings_op = bilm(context_character_ids)
elmo_context_input = weight_layers('input', context_embeddings_op, l2_coef=0.0)

Instructions for updating:
Use the `axis` argument instead
USING SKIP CONNECTIONS


In [5]:
count = -1
for i in train_files:
    with open(i) as f:
        for s in f:
            count = count + 1
            diag.append(s.split("|")[0].split(",")) 

            
    

In [6]:
import itertools
flat_diag = list(itertools.chain(*diag))
events = np.unique(flat_diag)
events = np.ndarray.tolist(events)

In [7]:
passed = ['d_041']

In [8]:
count = -1
for i in train_files:
    with open(i) as f:
        for s in f:
            count = count + 1
            sentences.append(s.split("|")[2].split(" ") +
                             s.split("|")[3].replace("\n", "").split(" "))


In [9]:
import itertools
flat_sentences = list(itertools.chain(*sentences))
events = np.unique(flat_sentences)
events = np.ndarray.tolist(events)

In [10]:
pred = 'lasso'

if pred == 'lasso':
    path_auc = '/users/emily/Documents/BERT_lasso_AUC'
    path_f1 = '/users/emily/Documents/BERT_lasso_F1'
    

if pred == 'DL':
    path_auc = '/users/emily/Documents/BERT_DL_AUC'
    path_f1 = '/users/emily/Documents/BERT_DL_F1'

AUC = []
F1 = []

for q in range(len(passed)):  
    AUC_hold=[]
    F1_hold = []

    for w in range(3):
        

        def prep_data(data_files):
            disease_prev = []
            omit1 = []
            sentences = []
            count = -1
            disease_prev = []
            omit1 = []
            count = -1
            for i in data_files:
                with open(i) as f:
                    for s in f:
                        count = count+1
                        age = float(s.split("|")[1].split(" ")[3].replace(",",r""))
                        if age <= 5:
                            omit1.append(1)
                        else:
                            omit1.append(0)
                            
                        prev_diags = [e for e in s.split("|")[2].split(" ") if e.startswith("d_")]
                        if passed[q] in prev_diags: # if disease exit in the previous diagnosis
                            disease_prev.append(1)
                        else:
                            disease_prev.append(0)
                        sentences.append(s.split("|")[2].split(" ") +
                                             s.split("|")[3].replace("\n", "").split(" "))
                        

            return sentences, omit1, disease_prev
        sentences, omit1, disease_prev = prep_data(train_files)
        


        ## Within each sentence, split medical record into individual visits  
        def split_sentences(sentences):
            newsents = []
            for count in range(len(sentences)):
                partials = []    
                z = 0
                for i in range(len(sentences[count])-1):        
                    i = i+1  
                    if (sentences[count])[i-1].startswith("d_"):             
                        if not sentences[count][i].startswith("d_") or (i == len(sentences[count])):
                            part = sentences[count][z:i]
                            partials.append(part)
                            z = i

                    if (sentences[count])[i-1].startswith("s"):
                        if not sentences[count][i].startswith("d_"): 
                            if not sentences[count][i].startswith("s") or (i == len(sentences[count])):
                                part = sentences[count][z:i]
                                partials.append(part)
                                z = i

                    if (sentences[count])[i-1].startswith("c"):
                        if not sentences[count][i].startswith("d_"):
                            if not sentences[count][i].startswith("s"): 
                                if not sentences[count][i].startswith("c") or (i == len(sentences[count])):
                                    part = sentences[count][z:i]
                                    partials.append(part)
                                    z = i

                newsents.append(partials)
            return newsents
        newsents = split_sentences(sentences)

        ## Within each sentence, split medical record into individual visits  
        def split_sentences2(sentences):
            newsents = []
            for count in range(len(sentences)):
                partials = []    
                z = 0
                for i in range(len(sentences[count])-1):        
                    i = i+1  
                    if (sentences[count])[i-1].startswith("d_"):             
                        if not sentences[count][i].startswith("d_") or (i == len(sentences[count])-1):
                            part = sentences[count][z:i]
                            partials.append(part)
                            z = i

                    if (sentences[count])[i-1].startswith("s"):
                        if not sentences[count][i].startswith("d_"): 
                            if not sentences[count][i].startswith("s") or (i == len(sentences[count])-1):
                                part = sentences[count][z:i]
                                partials.append(part)
                                z = i

                    if (sentences[count])[i-1].startswith("c"):
                        if not sentences[count][i].startswith("d_"):
                            if not sentences[count][i].startswith("s"): 
                                if not sentences[count][i].startswith("c") or (i == len(sentences[count])-1):
                                    part = sentences[count][z:i]
                                    partials.append(part)
                                    z = i

                newsents.append(partials)
            return newsents
        newsents2 = split_sentences2(sentences)

        #target_loc = []
        def adjust_all(newsents):
            # Identify the visit in the history where the target diagnosis occurs, if it does                                    
            target_loc = []
            ct1 = 0
            ct2 = 0
            for i in range(len(sentences)):
                ct = 0
                if passed[q] in list(np.concatenate(newsents[i])):
                    for j in range(len(newsents[i])):
                        if ct == 0 and passed[q] in (newsents[i])[j]:
                            target_loc.append(j)
                            ct = ct + 1
                            ct1 = ct1+1
                else:
                    target_loc.append(100)
                    ct2 = ct2+1
            return target_loc
        target_loc = adjust_all(newsents2)

        #data refers to train or test 
        def adjust_exclusions(sentences, newsents,omit):                    
            # create a vector indicating which patients are positive for target diagnosis

            disease_data = []
            for count in range(len(sentences)):  
                if passed[q] in sentences[count]:
                    disease_data.append(1)
                else:
                    disease_data.append(0)


            #exclude patients with target diagnosis present in first visit
            exc = []                
            for ii in range(len(sentences)): 
                if target_loc[ii]==0:
                    exc.append(1)
                else:
                    exc.append(0)

            #Update vector containing patients to exclude
            for count in range(len(sentences)):
                if exc[count]==1:
                    omit[count] = 1

            return disease_data, omit

        disease_train, omit1 = adjust_exclusions(sentences,newsents,omit1)


    #Convert events in history to patient vector


        def patient_vec(data_files, newsents, disease_prev, target,omit):
            count = -1
            patient_seq_all_disease = []
            exclude = []
 
            for i in data_files: 
                with open(i) as f:
                    for line in f:
                        count = count + 1
                        patient_seq = []
                        #feed_events = line.split("|")[2].split(" ")
                        if omit[count]==1 or newsents[count]==[]:
                            exclude.append(1)
                        else:
                            if disease_prev[count] == 1:  
                                visits = newsents[count][:target[count]]
                                if visits == []:
                                    exclude.append(1)
                                else:
                                    exclude.append(0)
                                    feed_events = list(np.concatenate(visits))

                                    
                                    te = len(feed_events)
                                    
                                    
                                    
                            
                                    weighted_events = [(e,  math.exp(decay*(j-te+1)/te)) for j, e in enumerate(newfeed) if e in events]
                                    sum_weights = sum(weight for event,weight in weighted_events)
                                    
                                    with tf.Session() as sess:
                                        # It is necessary to initialize variables once before running inference.
                                        sess.run(tf.global_variables_initializer())

                                        # Create batches of data.
                                        context_ids = batcher.batch_sentences([feed_events])


                                        # Compute ELMo representations (here for the input only, for simplicity).
                                        elmo_context_input_ = sess.run(
                                            elmo_context_input['weighted_op'],
                                            feed_dict={context_character_ids: context_ids}
                                        )


                                    flat_embeddings = list(itertools.chain(*elmo_context_input_))
                                    for a in weighted_events:
                                        event, weight = a
                                        patient_seq.append(weight*flat_embeddings[feed_events.index(event)])
                                    patient_seq_all_disease.append(sum(patient_seq)/sum_weights)

                            else:
                                visits = newsents[count]  
                                if visits == []:
                                    exclude.append(1)
                                else:
                                    feed_events = list(np.concatenate(visits))
                                    exclude.append(0)
                                    
                                    te = len(feed_events)
                                    

                                    
                                    
                                    weighted_events = [(e,  math.exp(decay*(j-te+1)/te)) for j, e in enumerate(newfeed) if e in events]
                                    sum_weights = sum(weight for event,weight in weighted_events)
                                                                        with tf.Session() as sess:
                                        # It is necessary to initialize variables once before running inference.
                                        sess.run(tf.global_variables_initializer())

                                        # Create batches of data.
                                        context_ids = batcher.batch_sentences([feed_events])


                                        # Compute ELMo representations (here for the input only, for simplicity).
                                        elmo_context_input_ = sess.run(
                                            elmo_context_input['weighted_op'],
                                            feed_dict={context_character_ids: context_ids}
                                        )


                                    flat_embeddings = list(itertools.chain(*elmo_context_input_))
                                    for a in weighted_events:
                                        event, weight = a
                                        patient_seq.append(weight*flat_embeddings[feed_events.index(event)])
                                    patient_seq_all_disease.append(sum(patient_seq)/sum_weights)
            return patient_seq_all_disease, exclude



        #Update labels for presence of target diagnosis
        def update_data(exclude,disease_data,data_disease):                
            disease_final = []
            for i in range(0,len(exclude)):
                if exclude[i] == 0:
                    disease_final.append(disease_data[i])

            data_disease[passed[q]] = disease_final

            X_data_disease=data_disease.iloc[:,0:100]
            Y_data_disease =data_disease.iloc[:,100]


            return X_data_disease, Y_data_disease

    


        patient_seq_all_disease, exclude= patient_vec(train_files, newsents,  disease_prev,target_loc,omit1)

        data_disease=pd.DataFrame(patient_seq_all_disease)

        X_train_disease, Y_train_disease = update_data(exclude,disease_train,data_disease)


        sentences, omit1, disease_prev = prep_data(valid_files)

        newsents = split_sentences(sentences)

        newsents2 = split_sentences2(sentences)

        target_loc = adjust_all(newsents2)

        disease_test, omit1 = adjust_exclusions(sentences, newsents,omit1)
              
        patient_seq_all_disease_test, exclude = patient_vec(valid_files, newsents,disease_prev, target_loc,omit1)
      
        data_disease_test = pd.DataFrame(patient_seq_all_disease_test)

              
        X_disease_test, Y_disease_test = update_data(exclude,disease_test,data_disease_test)

        if pred =='lasso':

            searchCV_lasso = LogisticRegressionCV(Cs=list(np.power(10.0, np.arange(-10, 10))),penalty='l1',
            cv=5,fit_intercept=True,solver='liblinear',max_iter = 10000,scoring="f1")
            lasso_fit_disease = searchCV_lasso.fit(X_train_disease,Y_train_disease)
            lasso_prob_disease = lasso_fit_disease.predict_proba(X_train_disease)[:,1]

            lasso_prob_disease = lasso_fit_disease.predict_proba(X_disease_test)[:,1]
            auc_disease_lasso = metrics.roc_auc_score(Y_disease_test, lasso_prob_disease)
            AUC_hold.append(auc_disease_lasso)
            F1_hold.append(f1_score(Y_disease_test, lasso_fit_disease.predict(X_disease_test)))


        if pred == 'DL':

            print("Reading files..."),
            tr_dataframe = data_disease
            te_dataframe = data_disease_test

            
            

            print("Fineshed\n")

            print("Compressing data..."),
            tr_X = tr_dataframe.iloc[:, :-1].values
            tr_Y = tr_dataframe.iloc[:, -1].values
            te_X = te_dataframe.iloc[:, :-1].values
            te_Y = te_dataframe.iloc[:, -1].values

            


            # model_DL = KerasRegressor(build_fn = baseline_model(units = Grid.best_params_['units'], act = Grid.best_params_['act']), nb_epoch=2000, batch_size=300,verbose = 0)

            model_DL = Sequential()
            model_DL.add(Dense(55,activation = 'relu'))
            model_DL.add(Dense(20,activation='relu'))
            model_DL.add(Dense(1,activation = 'sigmoid'))
            model_DL.compile(loss='mean_squared_error', optimizer='adam')



            # In[ ]:




            # In[32]:


            model_DL.fit(tr_X,tr_Y,batch_size = 100, epochs=70, verbose = 0)
            tr_predict_Y = model_DL.predict(tr_X)
            te_predict_Y = model_DL.predict(te_X)    






            # ROC AUC
            auc = metrics.roc_auc_score(te_Y, te_predict_Y)
            print('ROC AUC: %f' % auc)
            yhat_classes = model_DL.predict_classes(te_X, verbose=0)
            f1 = f1_score(te_Y, yhat_classes)
            print('F1: %f' % f1)

            AUC_hold.append(auc)
            F1_hold.append(f1)

    AUC.append(np.mean(AUC_hold))
    F1.append(np.mean(F1_hold))
   


from pandas import DataFrame
df_auc = DataFrame(AUC)
df_auc.to_csv(path_auc)
df_f1 = DataFrame(F1)
df_f1.to_csv(path_F1)



   


KeyboardInterrupt: 

In [ ]:
#elmo_data = pd.DataFrame([passed, AUC_logistic_elmo, AUC_lasso_elmo,AUC_DL_elmo,F1_logistic_elmo,F1_lasso_elmo,F1_DL_elmo, prevalence], index = ["Disease", "AUC Logistic","AUC lasso", 
                                                                                                             "AUC DL","F1 Logistic","F1 lasso","F1 DL","Prevalence"])
#elmo_data
#elmo_data.to_excel('/home/egetzen/elmo_data.xlsx') 